순환신경망(RNN: Recurrent Neural Network)
음성인식 또는 문장번역시 사용가능 / 시계열데이터 예측 가능
주요개념 : Embedding, SimpleRNN, LSTM, Conv1D

In [1]:
# Embedding층 : 단어(또는 데이터)를 벡터형태로 표현할 수 있음 -> 텍스트 분류의 가장 기본적인 층
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

texts = ['You are the Best', 'You are the Nice']
tokenizer = Tokenizer(num_words = 10, oov_token = '<OOV>')
tokenizer.fit_on_texts(texts)

# 텍스트 데이터를 정수 인덱스 형태로 변환
sequences = tokenizer.texts_to_sequences(texts)

# 이진형태로 인코딩
binary_results = tokenizer.sequences_to_matrix(sequences, mode = 'binary')

print(tokenizer.word_index)
print('------------------------------------------------------------------')

print(f'sequences: {sequences}\n')
print((f'binary_vectors:\n {binary_results}\n'))

test_text = ['You are the One']
test_sq = tokenizer.texts_to_sequences(test_text)

print(f'test sequences: {test_sq}')

{'<OOV>': 1, 'you': 2, 'are': 3, 'the': 4, 'best': 5, 'nice': 6}
------------------------------------------------------------------
sequences: [[2, 3, 4, 5], [2, 3, 4, 6]]

binary_vectors:
 [[0. 0. 1. 1. 1. 1. 0. 0. 0. 0.]
 [0. 0. 1. 1. 1. 0. 1. 0. 0. 0.]]

test sequences: [[2, 3, 4, 1]]


In [2]:
from tensorflow.keras.datasets import imdb

num_words = 10000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = num_words)

<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
C:\Users\Bestc\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\datasets\imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
C:\Users\Bestc\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\datasets\imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or

In [3]:
# 데이터 형태확인
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(25000,) (25000,)
(25000,) (25000,)


In [4]:
# imdb데이터는 이미 인코딩이 완료되어 있음.
# 데이터를 동일한 길이로 맞추기
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_len = 500
print('Before pad_sequences: ', len(X_train[0]))

pad_X_train = pad_sequences(X_train, maxlen = max_len, padding = 'pre')
pad_X_test = pad_sequences(X_test, maxlen = max_len, padding = 'pre')

print('After pad_sequences: ', len(pad_X_train[0]))

Before pad_sequences:  218
After pad_sequences:  500


In [5]:
# 모델 구성하기
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

model = Sequential()
# 해당층은 모델의 첫번째 층으로만 사용가능
# Flatten층을 사용하기 위해 input_length를 전달
# input_dim의 인자는 학습 데이터 셋에 사용한 단어의 개수
# output_dim의 인자는 임베딩 벡터의 크기
model.add(Embedding(input_dim = num_words, output_dim = 32, input_length = max_len))
model.add(Flatten())
model.add(Dense(1, activation = 'sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 32)           320000    
_________________________________________________________________
flatten (Flatten)            (None, 16000)             0         
_________________________________________________________________
dense (Dense)                (None, 1)                 16001     
Total params: 336,001
Trainable params: 336,001
Non-trainable params: 0
_________________________________________________________________


In [6]:
history = model.fit(pad_X_train, y_train, batch_size = 32, epochs = 5, validation_split = 0.2)

Epoch 1/5
625/625 [==============================] - 5s 8ms/step - loss: 0.4576 - acc: 0.7735 - val_loss: 0.2966 - val_acc: 0.8796
Epoch 2/5
625/625 [==============================] - 5s 8ms/step - loss: 0.2014 - acc: 0.9271 - val_loss: 0.2733 - val_acc: 0.8848
Epoch 3/5
625/625 [==============================] - 4s 7ms/step - loss: 0.1052 - acc: 0.9703 - val_loss: 0.2913 - val_acc: 0.8868
Epoch 4/5
625/625 [==============================] - 4s 7ms/step - loss: 0.0500 - acc: 0.9913 - val_loss: 0.3134 - val_acc: 0.8876
Epoch 5/5
625/625 [==============================] - 4s 7ms/step - loss: 0.0236 - acc: 0.9976 - val_loss: 0.3411 - val_acc: 0.8820


In [7]:
model.evaluate(pad_X_test, y_test)

782/782 [==============================] - 1s 2ms/step - loss: 0.3447 - acc: 0.8775


[0.3446876108646393, 0.8775200247764587]